In [17]:
import pandas as pd
import os

In [18]:
# Specify the data folder
data_folder = "./../processed_data"

# Read each CSV file into a separate DataFrame
df = pd.read_csv(os.path.join(data_folder, 'games.csv'))

# Baseline

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np



mae = mean_absolute_error(df["weekly_attendance"], df["avg_season_attendance_prev"])
rmse = mean_squared_error(df["weekly_attendance"], df["avg_season_attendance_prev"], squared=False)
mdae = np.median(np.abs(df["weekly_attendance"] - df["avg_season_attendance_prev"]))

print(f"MAE: {mae:.2f}")
print(f"MDAE: {mdae:.2f}")
print(f"RMSE: {rmse:.2f}")


ValueError: Input contains NaN.

In [21]:
# Check for NaN values in each column of df
# Check for NaN values in each column of df
nan_counts = df.isna().sum()
print(nan_counts[nan_counts > 0])

# Select rows with any NaN values
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)

avg_season_attendance_prev    16
home_team_wins_last_3          3
away_team_wins_last_3          3
zscore_pts_home               16
zscore_yds_home               16
zscore_turnovers_home         16
zscore_pts_away               16
zscore_yds_away               16
zscore_turnovers_away         16
dtype: int64
      year  week             home_team             away_team  day  \
1549  2016     1   San Francisco 49ers      Los Angeles Rams  Mon   
1562  2016     2      Los Angeles Rams      Seattle Seahawks  Sun   
1578  2016     3  Tampa Bay Buccaneers      Los Angeles Rams  Sun   
1592  2016     4     Arizona Cardinals      Los Angeles Rams  Sun   
1611  2016     5      Los Angeles Rams         Buffalo Bills  Sun   
1617  2016     6         Detroit Lions      Los Angeles Rams  Sun   
1641  2016     7      Los Angeles Rams       New York Giants  Sun   
1664  2016     9      Los Angeles Rams     Carolina Panthers  Sun   
1676  2016    10         New York Jets      Los Angeles Rams  Sun   
